In [3]:
!pip install wget


  Using cached wget-3.2.zip (10 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: wget is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for wget ... done

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 6.4 MB/s eta 0:00:0000:0100:01
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 6.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 9.1 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached wheel-0.38.4-py3-none-any.whl (36 kB)
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 20.1 MB/s eta 0:00:00
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-n

In [2]:
import torch
import torch.nn as nn

/Users/kyeongchanlee/.pyenv/versions/3.9.9/envs/candidate/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [74]:
t = torch.tensor([
   [[1, 2, 3, 4, 5],
   [4, 5, 6, 7, 8],
   [9, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],

   [[11, 12, 13, 14, 15],
   [14, 15, 16, 17, 18],
   [19, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
                
   [[21, 22, 23, 24, 25],
   [24, 25, 26, 27, 28],
   [29, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]]
   ])

In [75]:
t

tensor([[[ 1,  2,  3,  4,  5],
         [ 4,  5,  6,  7,  8],
         [ 9,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9]],

        [[11, 12, 13, 14, 15],
         [14, 15, 16, 17, 18],
         [19, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]],

        [[21, 22, 23, 24, 25],
         [24, 25, 26, 27, 28],
         [29, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]]])

In [76]:
t.size()

torch.Size([3, 4, 5])

In [90]:
class A:
    a = 10

In [91]:
A.a

10

In [101]:
class PrintInputShape:
    flag = True

    @classmethod
    def print_shape(self, t):
        if self.flag:
            ws = ''
            print(f"{ws}\\")
            ws += ' '
            print(f"{ws}{t.size()[0]}(0th dim)")
            ws += ' '
            print(f"{ws}\\")


            for line, _ in enumerate(t[-1], start=1):
                if line == 1:
                    s = '^'
                    sample = f"{_[0]}, {_[1]}, ..., {_[-1]}"
                elif line == len(t[-1]):  # 마지막줄
                    s = 'v'
                    sample = f"{_[0]}, {_[1]}, ..., {_[-1]}"
                else:
                    s = '|'
                    sample = f"\t\t."
                print(f"{ws}{s} {sample}")

                if line == len(t[-1])//2:
                    print(f"{t.size()[1]}(1th dim)")


            print(f"{ws}<-- {t.size()[2]}(2th dim) -->")

        self.flag = False



In [103]:
PrintInputShape.print_shape(t)

In [15]:
print(f"input shape : {t.shape}")
for dim in range(len(t.size())):
    print(f"dim : {dim}")
    for a in t[0]:



AttributeError: 'torch.Size' object has no attribute 'items'